In [1]:
import pandas as pd
import os
import itertools
from pyspark.sql import SparkSession
import time
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
import uuid



In [ ]:
DATA_DIR = "/home/jovyan/data"
RESULTS_DIR = "/home/jovyan/results/classification"

In [3]:
# clean up the results directory
if os.path.exists(RESULTS_DIR):
    for item in os.listdir(RESULTS_DIR):
        item_path = os.path.join(RESULTS_DIR, item)
        if os.path.isfile(item_path):
            os.remove(item_path)
        elif os.path.isdir(item_path):
            import shutil
            shutil.rmtree(item_path)


In [4]:
# .config(("spark.driver.maxResultSize", "4g")) \
# Create a Spark session
spark = SparkSession.builder \
    .appName("SparkLightgbmClassification") \
    .master(os.environ.get("SPARK_MASTER", "spark://spark-master:7077")) \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "1g") \
    .config("spark.python.worker.faulthandler.enabled", "true") \
    .config("spark.sql.execution.pyspark.udf.faulthandler.enabled", "true") \
    .getOrCreate()

# get spark context
sc = spark.sparkContext

print(f"Spark version: {spark.version}")
print(f"Spark UI available at: {spark.sparkContext.uiWebUrl}")

Spark version: 4.0.0-preview2
Spark UI available at: http://2309e119b7ae:4040


In [5]:
# get unique fold ids from the spark DataFrame
fold_ids = [1, 2, 3, 4]

fold_ids

[1, 2, 3, 4]

In [6]:
def train_lightgbm_model(params=None):
    """
    Train a LightGBM classification model with the specified parameters.
    
    Parameters:
    -----------
    params : dict, optional
        Parameters for LGBMClassifier. If None, default parameters will be used.
        
    Returns:
    --------
    str
        File path where results are stored
    """
    
    print(f"Training LightGBM classification model...{params}")
    # Default parameters if none provided
    if params is None:
        raise ValueError("No parameters provided for LightGBM model.")
    
    fold_id = params.pop("fold_id")

    test_df = pd.read_parquet(
        os.path.join(DATA_DIR, f"ts_fold_{fold_id}_test.parquet"),
    )
    train_df = pd.read_parquet(
        os.path.join(DATA_DIR, f"ts_fold_{fold_id}_train.parquet"),
    )

    train_x = train_df.drop(columns=["target", "date"])
    train_y = train_df["target"]
    test_x = test_df.drop(columns=["target", "date"])
    test_y = test_df["target"]

    # Initialize the classification model
    model = LGBMClassifier(**params)
    
    # Train the model and measure time
    start_time = time.perf_counter()
    model.fit(
        train_x, train_y,
        eval_set=[(test_x, test_y)],
        eval_metric='logloss',
        callbacks=[
            lgb.early_stopping(stopping_rounds=2000),
            lgb.log_evaluation(period=100),
        ]
    )
    end_time = time.perf_counter()
    training_time = end_time - start_time
    
    # Make predictions
    y_pred_proba = model.predict_proba(test_x)[:, 1]  # Probability of positive class
    y_pred = model.predict(test_x)  # Class predictions
    
    # Calculate classification metrics
    accuracy = accuracy_score(test_y, y_pred)
    precision = precision_score(test_y, y_pred)
    recall = recall_score(test_y, y_pred)
    f1 = f1_score(test_y, y_pred)
    auc = roc_auc_score(test_y, y_pred_proba)

    # Store detailed classification report as string
    class_report = classification_report(test_y, y_pred)
    print(f"\nClassification Report for fold {fold_id}:")
    print(class_report)

    results = {
        "fold_id": fold_id,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc,
        'training_time': training_time,
    }
    results.update(params)

    # generate uuid string for the results file name
    fp_id = str(uuid.uuid4())
    fp_name = os.path.join(RESULTS_DIR, f"results_fold_{fold_id}_{fp_id}.parquet")

    results_df = pd.DataFrame([results])
    results_df.to_parquet(
        fp_name,
        index=False,
    )

    return fp_name

In [7]:
# Define the parameter grid for hyperparameter tuning

param_grid = {
    "n_jobs": [2],
    "fold_id": fold_ids,
    "lambda_l1": [0.3, 12, 40],
    "max_depth": [10, 15, 17],
    "colsample_bytree": [0.1, 0.3, 0.4],
    "alpha": [0.2],
    "num_leaves": [2048,],
    "learning_rate": [0.03],
    "lambda_l2": [0.01, 0.1],
    "max_bin": [256,],
    "bagging_fraction": [1],
    "deterministic": [False],
    "objective": ["huber"],
    "metric": ["huber"],
    "n_estimators": [20000],
    "random_state": [42],
    "importance_type": ["gain"],
}


# Generate all combinations of parameters
param_keys = list(param_grid.keys())
param_values = list(param_grid.values())
param_combinations = list(itertools.product(*param_values))

# Create a list of dictionaries, each representing a specific combination
param_dicts = []
for combo in param_combinations:
    param_dict = dict(zip(param_keys, combo))
    param_dicts.append(param_dict)

# Display the number of combinations and the first few combinations
print(f"Total number of parameter combinations: {len(param_dicts)}")
print("\nFirst 3 parameter combinations:")
for i in range(min(3, len(param_dicts))):
    print(f"Combination {i+1}:")
    print(param_dicts[i])


Total number of parameter combinations: 216

First 3 parameter combinations:
Combination 1:
{'n_jobs': 2, 'fold_id': 1, 'lambda_l1': 0.3, 'max_depth': 10, 'colsample_bytree': 0.1, 'alpha': 0.2, 'num_leaves': 2048, 'learning_rate': 0.03, 'lambda_l2': 0.01, 'max_bin': 256, 'bagging_fraction': 1, 'deterministic': False, 'objective': 'huber', 'metric': 'huber', 'n_estimators': 20000, 'random_state': 42, 'importance_type': 'gain'}
Combination 2:
{'n_jobs': 2, 'fold_id': 1, 'lambda_l1': 0.3, 'max_depth': 10, 'colsample_bytree': 0.1, 'alpha': 0.2, 'num_leaves': 2048, 'learning_rate': 0.03, 'lambda_l2': 0.1, 'max_bin': 256, 'bagging_fraction': 1, 'deterministic': False, 'objective': 'huber', 'metric': 'huber', 'n_estimators': 20000, 'random_state': 42, 'importance_type': 'gain'}
Combination 3:
{'n_jobs': 2, 'fold_id': 1, 'lambda_l1': 0.3, 'max_depth': 10, 'colsample_bytree': 0.3, 'alpha': 0.2, 'num_leaves': 2048, 'learning_rate': 0.03, 'lambda_l2': 0.01, 'max_bin': 256, 'bagging_fraction': 1, 

In [8]:
process_these = param_dicts

rdd = sc.parallelize(process_these, numSlices=len(process_these))

rdd_result = rdd.map(lambda x: train_lightgbm_model(params=x)).collect()

print(f">>>>length of rdd_result: {len(rdd_result)}")


>>>>length of rdd_result: 216


In [11]:
df_list = [pd.read_parquet(fp) for fp in rdd_result]
results_df = pd.concat(df_list, ignore_index=True)

results_df.head

<bound method NDFrame.head of      fold_id  accuracy  precision    recall  f1_score   auc_roc  \
0          1   0.81640   0.965170  0.567481  0.714730  0.956283   
1          1   0.81325   0.967487  0.557982  0.707769  0.956144   
2          1   0.88485   0.969574  0.739082  0.838782  0.967396   
3          1   0.88735   0.969819  0.745250  0.842832  0.966985   
4          1   0.88930   0.966434  0.753022  0.846485  0.966572   
..       ...       ...        ...       ...       ...       ...   
211        4   0.77995   0.971112  0.467470  0.631129  0.961920   
212        4   0.76705   0.972710  0.433698  0.599914  0.963520   
213        4   0.76840   0.972913  0.437050  0.603153  0.963821   
214        4   0.78215   0.974339  0.471443  0.635428  0.963353   
215        4   0.77905   0.974419  0.463496  0.628187  0.962717   

     training_time  n_jobs  lambda_l1  max_depth  ...  learning_rate  \
0        19.079455       2        0.3         10  ...           0.03   
1        16.911805   

In [12]:
results_df[["training_time", "accuracy", "precision", "recall", "f1_score", "auc_roc"]].describe()

,training_time,accuracy,precision,recall,f1_score,auc_roc
count,216.000000,216.000000,216.000000,216.000000,216.000000,216.000000
mean,34.269994,0.823231,0.970075,0.581945,0.711175,0.964011
std,29.279173,0.069443,0.003745,0.178095,0.141305,0.005674
min,3.630724,0.686900,0.957959,0.234518,0.377782,0.949209
25%,12.991366,0.770362,0.967895,0.446280,0.612005,0.961591
50%,25.397932,0.815150,0.970882,0.562322,0.712312,0.965117
75%,46.395643,0.878587,0.972743,0.721545,0.828089,0.968310
max,133.229507,0.940150,0.975892,0.886755,0.922769,0.971967


In [13]:
spark.stop()